<a href="https://colab.research.google.com/github/chaiwencw/CPO-Price-Forecasting/blob/main/project_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd             #to deal with table
import numpy as np              #to deal with numbers
import datetime as dt           #to deal with datetime data
from collections import OrderedDict
from sklearn.preprocessing import PowerTransformer
#from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from SA_parameter_optimization import *

ModuleNotFoundError: ignored

In [ ]:
url = "https://raw.githubusercontent.com/jc-usm/FYP-USM_2021-2022/main/historical_FCPO.csv"
fcpo = pd.read_csv(url, na_values='-')
fcpo['Date'] = pd.to_datetime(fcpo['Date'], format = '%d/%m/%Y')
fcpo.info()

In [ ]:
fcpo['Volume (K)'] = fcpo['Volume (K)'].apply(lambda x: x*1000)
fcpo = fcpo.rename(columns = {'Volume (K)': 'Volume'})
median_volume = fcpo['Volume'].agg(np.median)
fcpo['Volume'] = fcpo['Volume'].fillna(median_volume)
fcpo.info()

In [ ]:
newdate = fcpo["Date"]

exact_traiding_date = pd.DataFrame({"year": newdate.dt.year,
              "month": newdate.dt.month,
              "day": newdate.dt.day,
              "hour": newdate.dt.hour,
              "dayofyear": newdate.dt.dayofyear,
              "weekofyear": newdate.dt.isocalendar().week,
              "dayofweek": newdate.dt.dayofweek,
              "weekday": newdate.dt.weekday,
              "quarter": newdate.dt.quarter,
             })


In [ ]:
fcpo_no = fcpo.drop(columns = ['Date'])
new_fcpo = pd.concat([exact_traiding_date, fcpo_no], axis = 1)
col_names = new_fcpo.columns
#new_fcpo_normalised = scaler.PowerTransformer().fit_transform(new_fcpo)
#new_fcpo_normalised = Normalizer().fit_transform(new_fcpo)
#scaled_fcpo = pd.DataFrame(new_fcpo_normalised, columns = col_names)

In [ ]:
#data = scaled_fcpo.drop("Close",axis=1)
#target = scaled_fcpo['Close']
#x_train, x_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, shuffle = False)

In [ ]:
data = new_fcpo.drop(columns = ['Close'])
target = np.array(new_fcpo['Close'])
target = target.reshape(len(target), 1)
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, shuffle = False)
scalerX = PowerTransformer().fit(x_train)
scalerY = PowerTransformer().fit(y_train)
x_train = scalerX.transform(x_train)
y_train = scalerY.transform(y_train)
x_test = scalerX.transform(x_test)
y_test = scalerY.transform(y_test)
y_train = np.ravel(y_train) 
y_test = np.ravel(y_test)

In [ ]:
# Parameters that are kept constant during the tuning process
const_param = dict()
const_param['kernel'] = 'rbf'

In [ ]:
# Parameter search space
param_dict = OrderedDict()
#size = 1000
#param_dict['degree'] = list(np.arange(1, 100)) #int type
#param_dict['gamma'] = [x for x in np.linspace(0, 100, num = size) if x > 0] #float - small
#param_dict['C'] = [x for x in np.linspace(0, 100, num = size) if x > 0] #float
#param_dict['kernel'] = ['rbf', 'sigmoid', 'linear']
param_dict['gamma'] = tuple(10**-x  for x in range(2, 21)) #max. python can accept is 32, where only these are larger than 0
param_dict['C'] = tuple(np.arange(10.0**4, 10.0**6 + 1)) # the larger the better
param_dict['epsilon'] = tuple(10**-x  for x in range(2, 21)) #float - small; smallest can get is 1E-32

In [ ]:
SA_parameters = {
    "param_dict": param_dict,
    "constant_params": const_param, 
    "X_train": x_train,
    "Y_train": y_train, 
    "X_valid": x_test,
    "Y_valid": y_test, 
    "ML_model": SVR,
    "no_iters": 250,
    "alpha": 0.95,
    "beta": 1, 
    "initial_temperature": 100,
    "min_temperature": 30,
    "result": True,
    "best_parameter": True,
    "total_iterations": True
}

In [ ]:
best_model, result, best_parameter, total_iter = simulate_annealing(**SA_parameters)

In [ ]:
best_parameter

In [ ]:
result

In [ ]:
preds = best_model.predict(x_test)
RegressionMetric(y_test, preds)

In [ ]:
t_model, predict = model_training(x_train, y_train, x_test, y_test, model_to_train = SVR, prediction = True)
RegressionMetric(y_test, predict)

In [ ]:
ytest = y_test.reshape(len(y_test), 1)
ytest = scalerY.inverse_transform(ytest)
ytest_df = pd.DataFrame(data = ytest, columns = ['Close'])
new_preds = preds.reshape(len(preds), 1)
ori_preds = scalerY.inverse_transform(new_preds)
preds_df = pd.DataFrame(data = ori_preds, columns = ['Prediction'])
df = pd.concat([ytest_df, preds_df], axis = 1)
df

In [ ]:
result.to_csv("result.csv", index = False)

In [ ]:
#ori_fcpo = PowerTransformer().fit(new_fcpo).inverse_transform(scaled_fcpo)
#ori_fcpo = pd.DataFrame(ori_fcpo, columns=col_names)
#ori_fcpo